In [2]:
import json 

basestr = "--gpu_ids 0 --shuffle_topo --run_test_freq 10 --ncf 16 16 16 16 16 16 --niter_decay 10 --niter 150 --resblocks 3 --lr 0.001 --drop_relu --num_threads 0 --selection_module --reweight_loss --dataset_mode intseg --arch meshcnn --batch_size 8  --which_epoch best --extrinsic_condition_placement pre --extrinsic_features onehot hks --edgefeatures dihedrals --distortion_metric arap --epoch_count 1 --resconv --leakyrelu --layernorm --cachefolder cache --anchorcachefolder anchorcache --delayed_distortion_epochs 0 --step2paramloss --export_save_path outputs --checkpoints_dir outputs --dropout --anchor_loss --max_sample_size 200 --max_dataset_size 5000 --load_pretrain --which_epoch latest --extrinsic_condition_placement pre --extrinsic_features onehot hks --edgefeatures dihedrals --network_load_path ./outputs/pretrain_aug --name 9_dcount_v2_dschedTrue_mixTrue_augFalse --cachefolder ftcache9 --anchorcachefolder ftanchorcache9 --export_save_path outputs/distortion_finetune --checkpoints_dir outputs/distortion_finetune --distortion_loss count_v2 --deltaschedule --delta 1000 --min_delta 1 --dataroot datasets/mixed --test_dir datasets/mixed/test --mixedtraining --supervised"
print(json.dumps(basestr.split()))

["--gpu_ids", "0", "--shuffle_topo", "--run_test_freq", "10", "--ncf", "16", "16", "16", "16", "16", "16", "--niter_decay", "10", "--niter", "150", "--resblocks", "3", "--lr", "0.001", "--drop_relu", "--num_threads", "0", "--selection_module", "--reweight_loss", "--dataset_mode", "intseg", "--arch", "meshcnn", "--batch_size", "8", "--which_epoch", "best", "--extrinsic_condition_placement", "pre", "--extrinsic_features", "onehot", "hks", "--edgefeatures", "dihedrals", "--distortion_metric", "arap", "--epoch_count", "1", "--resconv", "--leakyrelu", "--layernorm", "--cachefolder", "cache", "--anchorcachefolder", "anchorcache", "--delayed_distortion_epochs", "0", "--step2paramloss", "--export_save_path", "outputs", "--checkpoints_dir", "outputs", "--dropout", "--anchor_loss", "--max_sample_size", "200", "--max_dataset_size", "5000", "--load_pretrain", "--which_epoch", "latest", "--extrinsic_condition_placement", "pre", "--extrinsic_features", "onehot", "hks", "--edgefeatures", "dihedrals",

In [ ]:
import os 
from models.layers.meshing import Mesh 
from models.layers.meshing.io import PolygonSoup
import dill as pickle 
import numpy as np 
from pathlib import Path 
from util.util import clear_directory
import shutil
from models.layers.meshing.analysis import computeFaceAreas, computeDihedrals, computeHKS, computeVertexNormals, computeFaceNormals

# Construct mixed dataset 
synthratio = 0.25 
thingiratio = 0.75
synthdataset = "./datasets/synthetic_dataset"
thingidataset = "./datasets/thingi10k"
max_dataset_size = 5000

mixeddataset = "./datasets/mixed"

# Count total number of samples through labels
synthmeshes = {'train': [], 'test': []}
thingimeshes = {'train': [], 'test': []}
for folder in ['train', 'test']:
    count = 0 
    for labelfile in os.listdir(os.path.join(synthdataset, folder, "labels")):
        if labelfile.endswith(".npy"):
            count += 1
    print(f"Synth {folder}: {count}")
    
    for objfile in os.listdir(os.path.join(synthdataset, folder)):
        if objfile.endswith(".obj"):
            synthmeshes[folder].append(objfile)
    
    count = 0
    for objfile in os.listdir(os.path.join(thingidataset, folder)):
        if objfile.endswith(".obj"):
            thingimeshes[folder].append(objfile)
            
            with open(os.path.join(thingidataset, folder, "anchors", objfile.replace(".obj", "") + ".pkl"), 'rb') as f:
                anchors = pickle.load(f)
            count += len(anchors)
    print(f"Thingi {folder}: {count}")

# TODO: filter thingi meshes by size + hks calculation 
if os.path.exists(mixeddataset):
    clear_directory(mixeddataset)
    
for folder in ['train', 'test']:
    datadir = os.path.join(mixeddataset, folder)
    anchordir = os.path.join(mixeddataset, folder, 'anchors')
    Path(anchordir).mkdir(exist_ok=True, parents=True)
    
    count = 0
    while count < max_dataset_size:
        if np.random.uniform(0,1) <= synthratio: 
            labeldir = os.path.join(mixeddataset, folder, 'labels')
            Path(labeldir).mkdir(exist_ok=True, parents=True)
            
            # Randomly choose a synthetic mesh to use
            synthmesh = np.random.choice(synthmeshes[folder])
            meshname = synthmesh.replace(".obj", "")
            
            # Copy obj, labels, anchors
            shutil.copyfile(os.path.join(synthdataset, folder, synthmesh), 
                            os.path.join(datadir, synthmesh))
            shutil.copyfile(os.path.join(synthdataset, folder, "anchors", synthmesh.replace(".obj", "") + ".pkl"), 
                            os.path.join(anchordir, synthmesh.replace(".obj", "") + ".pkl"))
            
            with open(os.path.join(synthdataset, folder, "anchors", synthmesh.replace(".obj", "") + ".pkl"), 'rb') as f:
                anchors = pickle.load(f) 
            
            for i in range(len(anchors)):
                shutil.copyfile(os.path.join(synthdataset, folder, "labels", meshname + f"_{i}.npy"), 
                                os.path.join(labeldir, meshname + f"_{i}.npy"))
            count += len(anchors)
            
            synthmeshes[folder].remove(synthmesh)
        else:
            while True:
                # Randomly choose a synthetic mesh to use 
                thingimesh = np.random.choice(thingimeshes[folder])
                meshname = thingimesh.replace(".obj", "")
                
                ## Check mesh is good 
                soup = PolygonSoup.from_obj(os.path.join(thingidataset, folder, thingimesh)) 
                mesh = Mesh(soup.vertices, soup.indices)

                if len(mesh.topology.edges) > 20000:
                    print(f"Skipping too large mesh {thingimesh} ...")
                    continue 
                
                # Check if valid mesh 
                if mesh.topology.hasNonManifoldEdges():
                    print(f"Skipping nonmanifold mesh {thingimesh} ...")
                    continue
                
                try: 
                    mesh.topology.thorough_check()
                except Exception as e:
                    print(e) 
                    print(f"Failed thorough check mesh {thingimesh} ...")
                    continue 
                
                try:
                    vs, fs, es = mesh.export_soup() 
                except Exception as e:
                    print(e) 
                    print(f"Failed export mesh {thingimesh} ...")
                    continue 
                
                # Try computing HKS and heat geodesic
                try:
                    from igl import edge_lengths, heat_geodesic
                    from util.util import compute_hks
                    from util.diffusion_net.geometry import get_operators
                    import torch 
                    
                    scales = torch.logspace(-2, 0, 16)
                    vertices = torch.from_numpy(mesh.vertices).float()
                    faces = torch.from_numpy(np.ascontiguousarray(mesh.faces)).long()

                    frames, mass, L, evals, evecs, gradX, gradY = \
                        get_operators(vertices, faces, meshname, op_cache_dir=os.path.join(thingidataset, folder, "opcache"), overwrite_cache=True)
                    mesh.hks = compute_hks(evals, evecs, scales).detach().numpy()
                    
                    t = np.mean(edge_lengths(mesh.vertices, mesh.faces))**2
                    geodesics = heat_geodesic(mesh.vertices, mesh.faces, t, np.array([0]))
                except Exception as e:
                    print(e) 
                    print(f"Failed HKS/heat geodesic mesh {thingimesh} ...")
                    continue 
            
                # Check connected components 
                import pymeshlab 
                ms = pymeshlab.MeshSet()
                ms.clear() 
                ms.load_new_mesh(os.path.join(thingidataset, folder, thingimesh)) 
                topo = ms.get_topological_measures()
                
                # Components 
                components = topo['connected_components_number']
                if components > 1: 
                    print(f"Multiple components found for {thingimesh}")
                    continue 
                
                break 
                
            # Copy obj, labels, anchors
            shutil.copyfile(os.path.join(thingidataset, folder, thingimesh), 
                            os.path.join(datadir, thingimesh))
            shutil.copyfile(os.path.join(thingidataset, folder, "anchors", thingimesh.replace(".obj", "") + ".pkl"), 
                            os.path.join(anchordir, thingimesh.replace(".obj", "") + ".pkl"))
            with open(os.path.join(thingidataset, folder, "anchors", thingimesh.replace(".obj", "") + ".pkl"), 'rb') as f:
                anchors = pickle.load(f) 
            count += len(anchors)


In [1]:
# Filter meshes by edge counts 
from models.layers.meshing import Mesh 
from models.layers.meshing.io import PolygonSoup
import os 

datadir = "./datasets/thingi10k"
for folder in ['test', 'train']:
    for objfile in os.listdir(os.path.join(datadir, folder)):
        if not objfile.endswith(".obj"):
            continue 
        soup = PolygonSoup.from_obj(os.path.join(datadir, folder, objfile)) 
        mesh = Mesh(soup.vertices, soup.indices)
        
        if len(mesh.topology.edges) > 25000:
            print(f"Removing file {os.path.join(datadir, folder, objfile)} with edges {len(mesh.topology.edges)}...")
            os.remove(os.path.join(datadir, folder, objfile))

/net/projects/ranalab/guanzhi/.conda/envs/dawand/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
### Thingi10k finetuning 
basestr = "python train.py --gpu_ids 0 --shuffle_topo --run_test_freq 10 --ncf 16 16 16 16 16 16 " \
"--niter_decay 10 --niter 150 --resblocks 3 --lr 0.001 --drop_relu --num_threads 0 " \
"--selection_module --reweight_loss --dataset_mode intseg --arch meshcnn --batch_size 8  " \
"--which_epoch best --extrinsic_condition_placement pre --extrinsic_features onehot hks --edgefeatures dihedrals " \
"--distortion_metric arap --epoch_count 1 --resconv --leakyrelu --layernorm " \
"--cachefolder cache --anchorcachefolder anchorcache --delayed_distortion_epochs 0 --step2paramloss --export_save_path outputs " \
"--checkpoints_dir outputs --dropout " \
"--anchor_loss --max_sample_size 200 --max_dataset_size 5000 "


# Network specific 
basestr += "--load_pretrain --which_epoch latest --extrinsic_condition_placement pre --extrinsic_features onehot hks " \
    "--edgefeatures dihedrals --network_load_path ./outputs/pretrain_aug "
    
shape_file = open("./slurm/distortion_finetune", "w")
retrieve_shape_file = open("./slurm/distortion_finetune_net", "w")
render_shape_file = open("./slurm/distortion_finetune_render", "w")

# We have enough room to run every permutation of the above 
from itertools import product 
distortion_losses = ['count', 'count_v2', 'count_l0'] 
delta_schedule = [True, False] # countv2: 1000, count: min_delta = 0.01, delta = 2, count_l0: min_delta = 0.01, delta = 1
mixed = [True, False]
augs = [True, False]

settings = list(product(distortion_losses, delta_schedule, mixed, augs))
outdir = "distortion_finetune"

i = 0 
# Mixed jobs
for distortion_loss, delta_sched, mix, aug in settings:
    name = f"{i}_d{distortion_loss}_dsched{delta_sched}_mix{mix}_aug{aug}"
    full_str = basestr + f"--name {name} --cachefolder ftcache{i} --anchorcachefolder ftanchorcache{i} "
    full_str += f"--export_save_path outputs/{outdir} --checkpoints_dir outputs/{outdir} --distortion_loss {distortion_loss} "
    
    if delta_sched and distortion_loss == "count": 
        full_str += f"--deltaschedule --delta 2 --min_delta 0.01 "
    if delta_sched and distortion_loss == "count_v2": 
        full_str += f"--deltaschedule --delta 1000 --min_delta 1 "
    if delta_sched and distortion_loss == "count_l0": 
        full_str += f"--deltaschedule --delta 1 --min_delta 0.01 "
    
    if mix: 
        full_str += f"--dataroot datasets/mixed --test_dir datasets/mixed/test --mixedtraining --supervised "
    else:
        full_str += f"--dataroot ./datasets/thingi10k --test_dir ./datasets/thingi10k/test "
        
    # Run with augs
    if aug:
        full_str += "--num_aug 1 --rotaug --vnormaug --testaug "
        
    shape_file.write(full_str + "\n")
    retrieve_shape_file.write(f"python get_net_epoch.py --dir outputs/{outdir} --name {name}\n")
    render_shape_file.write(f"python render_preds.py --dir outputs/{outdir} --name {name} --render_uv\n")
    i += 1
            
shape_file.close()
retrieve_shape_file.close()
render_shape_file.close()


In [4]:
import json 

cmds = "--gpu_ids 0 --mode evaluation --shuffle_topo --export_view_freq 4 --run_test_freq 4 --ncf 16 16 16 16 16 16 --niter_decay 10 --niter 150 --resblocks 3 --lr 0.001 --drop_relu --num_threads 0 --save_latest_freq 10 --save_epoch_freq 2 --selection_module --reweight_loss --dataset_mode intseg --arch meshcnn --resconv --batch_size 8 --leakyrelu --layernorm --which_epoch best --extrinsic_condition_placement pre --extrinsic_features onehot hks --edgefeatures dihedrals --export_preds --distortion_metric arap --cachefolder cache --anchorcachefolder anchorcache --delayed_distortion_epochs 0 --step2paramloss --export_save_path outputs --checkpoints_dir outputs --supervised --dataroot ./datasets/thingi10k --test_dir ./datasets/thingi10k/test --anchor_loss --max_sample_size 200 --max_dataset_size 5000  --distortion_loss count_v2 --gcsmoothness --gcsmoothness_weight 0.001 --num_aug 1 --vnormaug --testaug --max_grad 0.1 --name grad0.1_losscount_v2_augTrue"
print(json.dumps(cmds.split()))

["--gpu_ids", "0", "--mode", "evaluation", "--shuffle_topo", "--export_view_freq", "4", "--run_test_freq", "4", "--ncf", "16", "16", "16", "16", "16", "16", "--niter_decay", "10", "--niter", "150", "--resblocks", "3", "--lr", "0.001", "--drop_relu", "--num_threads", "0", "--save_latest_freq", "10", "--save_epoch_freq", "2", "--selection_module", "--reweight_loss", "--dataset_mode", "intseg", "--arch", "meshcnn", "--resconv", "--batch_size", "8", "--leakyrelu", "--layernorm", "--which_epoch", "best", "--extrinsic_condition_placement", "pre", "--extrinsic_features", "onehot", "hks", "--edgefeatures", "dihedrals", "--export_preds", "--distortion_metric", "arap", "--cachefolder", "cache", "--anchorcachefolder", "anchorcache", "--delayed_distortion_epochs", "0", "--step2paramloss", "--export_save_path", "outputs", "--checkpoints_dir", "outputs", "--supervised", "--dataroot", "./datasets/thingi10k", "--test_dir", "./datasets/thingi10k/test", "--anchor_loss", "--max_sample_size", "200", "--ma

In [1]:
## Different NO pre-training training options 
import os 

# TODO: test these values 
max_grads = [None, 0.1, 1, 5]
smoothness = [None, "default", "anneal"]
inits = [None, 'anchor', 'neighbors']
augs = [True, False]
losses = ['count', 'count_v2']
datasets = ['pretrain', 'thingi10k']

basestr = "python train.py --gpu_ids 0 --mode evaluation --shuffle_topo --export_view_freq 4 --run_test_freq 4 --ncf 16 16 16 16 16 16 " \
"--niter_decay 10 --niter 150 --resblocks 3 --lr 0.001 --drop_relu --num_threads 0 --save_latest_freq 10 --save_epoch_freq 2 " \
"--selection_module --reweight_loss --dataset_mode intseg --arch meshcnn --resconv --batch_size 4 --leakyrelu --layernorm " \
"--which_epoch best --extrinsic_condition_placement pre --extrinsic_features onehot hks --edgefeatures dihedrals " \
"--export_preds --distortion_metric arap --epoch_count 1 " \
"--cachefolder cache --anchorcachefolder anchorcache --delayed_distortion_epochs 0 --step2paramloss --export_save_path outputs " \
"--checkpoints_dir outputs --supervised --dataroot ./datasets/thingi10k --test_dir ./datasets/thingi10k/test " \
"--anchor_loss --max_sample_size 200 --max_dataset_size 5000 --plot_preds "

f = open("slurm/nopretrain_jobs", 'w')
netf = open("slurm/nopretrain_jobs_net", 'w') 
## First test just the gradient clamping with the different loss types and augs 
for grad in max_grads:
    for loss in losses:
        for aug in augs:
            expstr = basestr + f" --distortion_loss {loss} --gcsmoothness --gcsmoothness_weight 0.001"
            if aug:
                expstr += " --num_aug 1 --vnormaug --testaug"
            if grad:
                expstr += f" --max_grad {grad}"
            name = f"grad{grad}_loss{loss}_aug{aug}"
            expstr += f" --name {name}"
            netstr =  f"python get_net_epoch.py --dir outputs --name {name}"
            f.write(expstr + "\n")
            netf.write(netstr + "\n")
f.close() 
netf.close()

# for dataest in datasets: 
#     for loss in losses: 
#         for init in inits: 
#             for smooth in smoothness: 
#                 for grad in max_grads:
#                     for aug in augs:
#                         if aug:
#                             expstr = basestr + f" --num_aug 1 --vnormaug --testaug " 
#                         else:
#                             expstr = basestr 

In [4]:
## Different pre-training options 
augs = [True, False]
basestr = "python train.py --gpu_ids 0 --mode evaluation --shuffle_topo --export_preds --export_view_freq 10 --run_test_freq 10 " \
    "--ncf 16 16 16 16 16 16 --niter_decay 10 --niter 150 --resblocks 3 --lr 0.001 --drop_relu --num_threads 0 --save_latest_freq 10 " \
    "--save_epoch_freq 2 --selection_module --reweight_loss --dataset_mode intseg --arch meshcnn --test_dir ./datasets/synthetic_dataset/test " \
    "--dataroot ./datasets/synthetic_dataset --checkpoints_dir outputs --max_sample_size 8000 --supervised --epoch_count 1 " \
    "--extrinsic_condition_placement pre --batch_size 15 --name pretrain --cachefolder cache --anchorcachefolder anchorcache --resconv " \
    "--batch_size 8 --leakyrelu --layernorm --dropout --extrinsic_features onehot hks --edgefeatures dihedrals --plot_preds "
base_netstr = "python get_net_epoch.py"

f = open("slurm/pretrain_jobs", 'w')
netf = open("slurm/pretrain_jobs_net", 'w') 
for aug in augs:
    if aug:
        expstr = basestr + f" --export_save_path outputs --num_aug 1 --vnormaug --testaug --name pretrain_aug" 
        netstr = base_netstr + f" --dir outputs --name pretrain_aug"
    else:
        expstr = basestr + f" --export_save_path outputs --name pretrain"
        netstr = base_netstr + f" --dir outputs --name pretrain_aug"
    f.write(expstr + "\n")
    netf.write(netstr + "\n")
f.close()
netf.close()